In [33]:
import redcap
import pandas as pd
from datetime import datetime, timedelta
import re


# REDCap API endpoint and token
data = {
    'token': 'DE12CA04415CF93BCD68321EBF5040FD',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
df= redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])

data = {
    'token': 'D6807C091B1FC14F57CC9B2972D0B398',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
indigo=redcap.Project('https://ruff.mrc.gm:8443/redcap/api/',data['token'])
data = {
    'token': 'B5C8DD01C47A66E81E154A8AC735D281',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
ck_wk=redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])


In [34]:
sen_data=ck_wk.export_records(forms=['enumeration_and_sensitisation'])
sen_log_data=pd.DataFrame(sen_data)
sen_log_data=sen_log_data[['wk_ckno','enu_name','enu_village', 'enu_mname', 'enu_fname','enu_comp','sen_contact1','sen_contact2','sen_contact3']]
sen_log_data=sen_log_data.rename(columns={'wk_ckno': 'ck_wkno'})
village_mapping = {
'001':	'DUMBUTO',
'002':	'SANKANDI',
'003':	'NIORO JATTABA',
'004':	'JATTABA',
'005':	'JIFFARONG',
'006':	'BAJANA',
'007':	'KULI KUNDA',
'008':	'JAMARU',
'009':	'BRIKAMANDING',
'010':	'KANTONG KUNDA',
'011':	'JALI',
'013':	'MANDUAR',
'014':	'BANG KULING',
'015':	'GISSAY',
'016':	'TANKULAR',
'017':	'JOLI',
'018':	'KUYANG',
'019':	'BANTASU',
'020':	'SANTAMBA',
'021':	'MISSIRA',
'022':	'TABORANGKOTO',
'023':	'BURONG',
'024':	'JULA KUNDA',
'025':	'KARANTABA',
'026':	'MANDINA',
'027':	'JANNEH KUNDA',
'028':	'KEMOTO',
'029':	'KENEBA',
'030':	'BATELLING',
'031':	'SANDENG',
'032':	'WUDEBA',
'034':	'KENOKOTO',
'035':	'MANARI',
'036':	'NINETEEN',
'040':	'WUROKANG',
'041':	'KWINELLA SANSANKONO',
'042':	'KWINELLA NIA KUNDA',
'043':	'TENDABA',
'044':	'BUMARR',
'045':	'BAMBAKO',
'046':	'KUNDONG MARIAYA',
'047':	'NEMA',
'048':	'KUNDANG NUMU KUNDA',
'049':	'KUNDANG FULA KUNDA',
'050':	'NEMA KUTA',
'051':	'JIRROFF',
'052':	'MADINA ANGALLEH',
'053':	'JATTA KUNDA',
'054':	'MANDINA CENTRAL',
'055':	'SARE SARJO',
'056':	'SIBETO',
'057':	'SARE NDALLA',
'058':	'TABANANI',
'060':	'WILLINGARA',
'061':	'SARE MAMUDU',
'070':	'KOLIOR',
'071':	'JOMARR',
'072':	'JASOBO',
'073':	'SARE MUSA',
'074':	'YORRO JULA',
'075':	'BABOU YAA',
'076':	'MASEMBEH',
'077':	'GENIERE',
'078':	'KAIAF',
'079':	'MADINA KAIAF',
'080':	'SARE SAMBA',
'081':	'MADINA CEESAY KUNDA',
'082':	'NJOLFEN',
'083':	'TORANKA BANTANG',
'084':	'SAREH PATEH',
'085':	'MUNKUTALA'
}
# DataFrame
sen_log_data['village_name'] = sen_log_data['enu_village'].map(village_mapping)
sen_log_data=sen_log_data[
    (sen_log_data['ck_wkno']!='')
]

In [35]:
df_consent=df.export_records(forms=['consent'])
df_consent=pd.DataFrame(df_consent)
df_consent=df_consent[
    (df_consent['ck_wkno']!='')
]
df_consent=df_consent[['participant_id','ck_wkno']]

In [36]:
#eden deveries 
df_eden_deliveries=df.export_records(forms=['delivery'])
df_eden_deliveries=pd.DataFrame(df_eden_deliveries)
df_eden_deliveries=df_eden_deliveries[
    (df_eden_deliveries['del_ddate_q7']!='')&
    (df_eden_deliveries['del_outcome_q9']=='1')
    
]
df_eden_deliveries=df_eden_deliveries[['participant_id','del_outcome_q9','del_ddate_q7']]



In [37]:
#indigo _scann
df_indigo_scann=indigo.export_records(forms=['consent','ultrasound_scan'])
df_indigo_scann=pd.DataFrame(df_indigo_scann)
df_indigo_scann = df_indigo_scann[
    (df_indigo_scann['con_eden_participant_q10']=='1')&
    (df_indigo_scann['pregnancy_confirmed_by_ult_q2'])
]
#eden indigo numbers 
df_eden_indigo_id=df_indigo_scann[['con_participantid_q1','con_participant_eden_num_q11']]


In [38]:
df_indigo_deliveries=indigo.export_records(forms=['delivery'])
df_indigo_deliveries=pd.DataFrame(df_indigo_deliveries)

df_indigo_deliveries=df_indigo_deliveries[
    (df_indigo_deliveries['del_ddate_q7']!='')
]
df_indigo_deliveries=df_indigo_deliveries[['con_participantid_q1','del_outcome_q9','del_ddate_q7']]
df_indigo_deliveries=pd.merge(df_eden_indigo_id,df_indigo_deliveries,on='con_participantid_q1',how='inner')

df_indigo_deliveries=df_indigo_deliveries.rename(columns={'con_participant_eden_num_q11':'participant_id'})
# Eden Indigo deliveries
df_eden_indigo_deveries = pd.concat([df_indigo_deliveries, df_eden_deliveries], ignore_index=True).drop_duplicates('participant_id')


In [39]:
# withdrawal 
df_withdrawal=df.export_records(forms=['withdrawal'])
df_withdrawal=pd.DataFrame(df_withdrawal)
df_withdrawal = df_withdrawal[
    (df_withdrawal['wd_vdadte_q1']!= '')
]

In [40]:
# seasonal blood collection 
df_seasonal_blood_collection=df.export_records(forms=['seasonal_blood_collection_2'])
df_seasonal_blood_collection=pd.DataFrame(df_seasonal_blood_collection)
df_seasonal_blood_collection=df_seasonal_blood_collection[
    (df_seasonal_blood_collection['sbc_date']!='')
]
df_seasonal_blood_collection=df_seasonal_blood_collection[['participant_id','sbc_date','sbc_available','sbc_season']]

In [41]:
df_seasonal_blood_collection1=df_eden_indigo_deveries[~df_eden_indigo_deveries['participant_id'].isin(df_withdrawal['participant_id'])]
df_seasonal_blood_collection2=pd.merge(df_seasonal_blood_collection1,df_seasonal_blood_collection,on='participant_id',how='left').drop_duplicates('participant_id')
df_seasonal_blood_collection3=pd.merge(df_seasonal_blood_collection2,df_consent,on='participant_id',how='inner').drop_duplicates('participant_id')
df_seasonal_blood_collection4=pd.merge(sen_log_data,df_seasonal_blood_collection3,on='ck_wkno',how='right')